In [9]:
from kiutils.board import Board
from kiutils.libraries import LibTable
from kiutils.schematic import Schematic
from kiutils.footprint import Footprint
from kiutils.symbol import SymbolLib
from kiutils.wks import WorkSheet
from kiutils.dru import DesignRules


In [10]:
def get_symbol_by_entry_name(symbols, entry_name):
    for sym in symbols:
        if sym.entryName == entry_name:
            return sym
    return None

def get_pin_list_from_symbol(symbol):
    # Convert keys to int and find the maximum pin number
    try:
        pins_int = {int(number): uuid for number, uuid in symbol.pins.items()}
    except ValueError:
        raise ValueError("Pin numbers must be integers.")
    max_pin = max(pins_int.keys())

    # Create a list with None for missing pins
    pin_list = [None] * (max_pin + 1)

    # Fill the list so that index == pin number
    for number, uuid in pins_int.items():
        pin_list[number] = {"number": number, "uuid": uuid}

    # # Example: print the pins
    # for idx, pin in enumerate(pin_list):
    #     if idx == 18:
    #         print(f"Pin number: {pin['number']}, UUID: {pin['uuid']}")
    return pin_list

def get_pins_from_library(lib_file):
    # Load the library
    library = SymbolLib().from_file(lib_file)

    if len(library.symbols) != 1:
        raise ValueError("Library should contain exactly one symbol.")

    pin_list = []

    for sym in library.symbols:
        # Iterate over all units
        for unit in sym.units:
            # print(f"  Unit {unit.unitId}:")
            for pin in unit.pins:  # pins are SymbolPin objects
                pin_list.append(pin)
                # print(f"    Pin number: {pin.number}, name: {pin.name}, position: ({pin.position})")
    return pin_list

In [11]:
lib_file = "STM32H562VGT6.kicad_sym"
schematic_file = "SDIO_Multiplexer.kicad_sch"

#load schematic from file
schematic = Schematic().from_file(schematic_file)

#extract all symbols from schematic
symbols = schematic.schematicSymbols

#find the symbol based on its entryName
symbol = get_symbol_by_entry_name(symbols, "STM32H562VGT6")
if symbol.position.angle != 0:
    raise ValueError("Rotation not implemented yet")

pin_list = get_pin_list_from_symbol(symbol)


# Load the symbol library to the the pins location
pin_list = get_pins_from_library(lib_file)

# Adjust pin positions based on symbol position
for pin in pin_list:
    pin.position = (symbol.position.X + pin.position.X,  symbol.position.Y - pin.position.Y)


for pin in pin_list:
    if pin.number == '1' or pin.number == '3':
        print(f"Pin number: {pin.number}, Name: {pin.name}, Position: {pin.position}")

#find all labels
labels = schematic.labels


for label in labels:
    print(label.uuid, label.text, label.position)

wires_raw = [item for item in schematic.graphicalItems if item.type == "wire"]
wires = []
for wire_raw in wires_raw:
    # unpack start and end points
    points = wire_raw.points
    # Extract start coordinates (first point)
    start_x = points[0].X
    start_y = points[0].Y

    # Extract end coordinates (second point)
    end_x = points[1].X
    end_y = points[1].Y
    wires.append({
        "uuid": wire_raw.uuid,
        "start": (start_x, start_y),
        "end": (end_x, end_y)
    })
#delete wires_raw  # free memory
del wires_raw, wire_raw, points, start_x, start_y, end_x, end_y
for wire in wires:
    print(f"Wire UUID: {wire['uuid']}, Start: {wire['start']}, End: {wire['end']}")

Pin number: 1, Name: PE2, Position: (163.83, 45.72)
Pin number: 3, Name: PE4, Position: (163.83, 50.8)
15af9977-595b-4391-8578-67ae5803b1b6 FSADDAFSF Position(X=179.07, Y=16.51, angle=270, unlocked=False)
7de97ee7-65ee-4337-a8e4-7fa829398291 TEST_LABEL Position(X=111.76, Y=45.72, angle=0, unlocked=False)
a0555489-99d8-464f-8895-506ebdddb569 GPIO23 Position(X=120.65, Y=57.15, angle=0, unlocked=False)
Wire UUID: 2a244414-ef94-473e-9295-ed3d9541b4f7, Start: (146.05, 57.15), End: (146.05, 50.8)
Wire UUID: 573ca34c-7035-4cad-8fa2-8f10d1f21200, Start: (179.07, 27.94), End: (179.07, 16.51)
Wire UUID: 797949d9-0969-435d-995b-aa31a3b9311d, Start: (120.65, 57.15), End: (146.05, 57.15)
Wire UUID: c844e704-aab4-4e1e-998d-4d4323534410, Start: (146.05, 50.8), End: (163.83, 50.8)
Wire UUID: f0e217db-ff03-44aa-9904-928cae79af4d, Start: (111.76, 45.72), End: (163.83, 45.72)


In [12]:
from collections import defaultdict, deque

TOL = 0.01  # coordinate tolerance

def is_close(p1, p2, tol=TOL):
    return abs(p1[0] - p2[0]) < tol and abs(p1[1] - p2[1]) < tol

def build_wire_graph(wires):
    """
    Build adjacency graph of wires. Wires are connected if they share a coordinate.
    Returns:
        wire_graph: dict wire_uuid -> set of connected wire_uuids
        coord_to_wires: dict coord -> list of wire_uuids touching it
    """
    coord_to_wires = defaultdict(list)
    for wire in wires:
        coord_to_wires[wire['start']].append(wire['uuid'])
        coord_to_wires[wire['end']].append(wire['uuid'])

    wire_graph = defaultdict(set)
    for wire in wires:
        wire_id = wire['uuid']
        for neighbor in coord_to_wires[wire['start']] + coord_to_wires[wire['end']]:
            if neighbor != wire_id:
                wire_graph[wire_id].add(neighbor)
    return wire_graph, coord_to_wires

def label_to_pin_map(pins, labels, wires, tol=TOL):
    """
    Map labels to pins through wire connectivity (multi-segment nets and junctions supported)
    Returns: dict label_text -> list of SymbolPin objects
    """
    wire_graph, coord_to_wires = build_wire_graph(wires)

    # Map pins for easy lookup
    pin_id_map = {id(pin): pin for pin in pins}

    # Map pins -> wires they touch
    pin_to_wires = defaultdict(set)
    for pin in pins:
        for coord, wire_uuids in coord_to_wires.items():
            if is_close(pin.position, coord, tol):
                pin_to_wires[id(pin)].update(wire_uuids)
    # Map labels -> wires they touch
    label_to_wires = defaultdict(set)
    for label in labels:
        label_pos = (label.position.X, label.position.Y)
        for coord, wire_uuids in coord_to_wires.items():
            if is_close(label_pos, coord, tol):
                label_to_wires[label.text].update(wire_uuids)
    # BFS function to find all reachable wires from a set of starting wires
    def reachable_wires(start_wires):
        visited = set()
        queue = deque(start_wires)
        while queue:
            w = queue.popleft()
            if w in visited:
                continue
            visited.add(w)
            for neighbor in wire_graph[w]:
                if neighbor not in visited:
                    queue.append(neighbor)
        return visited

    # Build label -> pins mapping
    label_to_pins = defaultdict(list)
    for label_text, start_wires in label_to_wires.items():
        all_reachable = reachable_wires(start_wires)
        for pin_id, pin_wires in pin_to_wires.items():
            if all_reachable & pin_wires:  # if any wire overlaps
                label_to_pins[label_text].append(pin_id_map[pin_id])
    return label_to_pins



# ------------------------
# Example usage
# pins = [...]    # list of SymbolPin objects
# labels = [...]  # list of LocalLabel objects
# wires = [...]   # list of dicts with 'uuid', 'start', 'end'

label_to_pins_map = label_to_pin_map(pin_list, labels, wires)
for label, connected_pins in label_to_pins_map.items():
    print(f"Label '{label}' connects to pins:")
    for pin in connected_pins:
        print(f"  Pin {pin.number} ({pin.name})")


Label 'FSADDAFSF' connects to pins:
  Pin 97 (PE0)
Label 'TEST_LABEL' connects to pins:
  Pin 1 (PE2)
Label 'GPIO23' connects to pins:
  Pin 3 (PE4)


In [3]:
from helper_functions.kicad_read_pins import kicad_pins_to_labels_map
lib_file = "STM32H562VGT6.kicad_sym"
schematic_file = "SDIO_Multiplexer.kicad_sch"
pins_to_labels = kicad_pins_to_labels_map(lib_file, schematic_file)

for label, connected_pins in pins_to_labels.items():
    print(f"Label '{label}' connects to pins:")
    for pin in connected_pins:
        print(f"  Pin {pin.number} ({pin.name})")

Label 'TEST_LABEL' connects to pins:
  Pin 1 (PE2)
Label 'GPIO23' connects to pins:
  Pin 3 (PE4)
